In [2]:
import numpy as np
import pandas as pd
import math
import zipfile
import copy
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.utils import shuffle
from mpl_toolkits.basemap import Basemap
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier as DT
from sklearn.feature_selection import chi2, VarianceThreshold, SelectKBest
from sklearn.metrics import accuracy_score

In [3]:
modelSVC1 = SVC(probability=True, 
            class_weight='balanced',
            shrinking=False,
            verbose=True)
modelSVC2 = SVC(probability=True,
            class_weight=None,
            shrinking=True,
            verbose=False)
modelLSVC1 = LinearSVC(dual=True,
            class_weight='balanced',
            loss='hinge',
            fit_intercept=True)
modelLSVC2 = LinearSVC(dual=False,
                       loss='squared_hinge', 
                       fit_intercept=True)
modelDT1 = DT(criterion="gini",
            splitter="best",
             class_weight="balanced")
modelDT2 = DT(criterion="entropy",
             splitter="random",
             class_weight=None)

In [4]:
models = {'decision tree' : modelDT2}

In [5]:
#Expects sampled data already
def preprocess(to_process):
    #remove 0s and nans
    to_process = to_process.loc[~(to_process==0).all(axis=1)]
    to_process.dropna()
    
    
    #convert X to positive (all are negative and it causes problems later on)
    to_process['X'] = to_process['X'].map(lambda x: abs(x))
    
    #Create new features from the date data
    to_process['Year'] = to_process['Dates'].map(lambda x: x.year)
    to_process['Week'] = to_process['Dates'].map(lambda x: x.week)
    to_process['Hour'] = to_process['Dates'].map(lambda x: x.hour)
    
    #drop descript and resolution because they aren't in the test set so they can't be used
    del to_process['Dates']
    del to_process['Descript']
    del to_process['Resolution']
    del to_process['Address']
    
    le = preprocessing.LabelEncoder()
    to_process['Category'] = le.fit_transform(to_process['Category'])
    to_process['DayOfWeek'] = le.fit_transform(to_process['DayOfWeek'])
    to_process['PdDistrict'] = le.fit_transform(to_process['PdDistrict'])
    
    return to_process

In [6]:
#Selects K best features using the featureSelector selector and verifying they fit a variance threshhold
def extract_features(to_process, to_process_predicted, featureSelector, threshold=.8, kBest=1):
    selector = VarianceThreshold() 
    selector.fit_transform(to_process, threshold)
    
    #fit the data to the VarianceThreshold
    selector = SelectKBest(featureSelector, k=kBest)
    selector.fit(to_process, to_process_predicted)
    
    # Get idxs of columns to keep
    idxs_selected = selector.get_support(indices=True)
    #Create new dataframe with only desired columns, or overwrite existing
    data_new = to_process[idxs_selected]
    
    return data_new

In [7]:
def model_kfolds(sampled_data, y_val, model_dict, kFolds=5):
    results = []
    
    #metrics to score with
    metrics = ['accuracy', 'f1_micro', 'precision_micro', 'recall_micro']#, 'precision', 'recall']
    
    for name, model in model_dict.items():
        #copy the data before each use
        data_copy = copy.deepcopy(sampled_data)
        
        
        scores = cross_validate(model, 
                                data_copy, 
                                data_copy[y_val], 
                                scoring=metrics, 
                                cv=kFolds)
        
        results.append([name, scores])
        
    return results

In [8]:
def pipeline(train_filename, test_filename, sample_size, kFolds=True):
    #get the data from the CSVs
    train = pd.read_csv(train_filename, parse_dates=['Dates'])
    test = pd.read_csv(test_filename, parse_dates=['Dates'])
    
    #sample data
    sampled_data = train.sample(n=sample_size)
    
    #preprocesses data by converting to nominal features, removing zeros,
    #deleting unusable features, creating new features, and modifying negatives
    processed_data = preprocess(sampled_data)
    
    cats = [np.array(data.Category), np.array(data.transformed_cat)]
    
    del processed_data['Category']
    
    #select K best features
    final = extract_features(processed_data, processed_data['Category'], chi2, threshold=.8, kBest=5)
    #print(final)
    
    
    
    #get models
    results = model_kfolds(final, "Category", models)
    
    
    #return performance metrics 
    results = [{x[0] : {'accuracy' : x[1]['test_accuracy'],
                        'precision' : x[1]['test_precision_micro'],
                        'recall' : x[1]['test_recall_micro'],
                        'f1' : x[1]['test_f1_micro']}} for x in results]
    
    return results

# Sampling

In [9]:
data = pd.read_csv('train.csv', parse_dates=['Dates'])

In [10]:
data_sample = data.sample(5000)

In [11]:
data_sample.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
579003,2007-03-07 07:58:00,OTHER OFFENSES,TRAFFIC VIOLATION,Wednesday,NORTHERN,"ARREST, CITED",OCTAVIA ST / FELL ST,-122.424437,37.775740
214827,2012-06-30 22:00:00,OTHER OFFENSES,FALSE PERSONATION,Saturday,SOUTHERN,NONE,800 Block of BRYANT ST,-122.403405,37.775421
600439,2006-11-04 14:00:00,VEHICLE THEFT,STOLEN AUTOMOBILE,Saturday,MISSION,UNFOUNDED,400 Block of CASTRO ST,-122.435150,37.761760
164619,2013-03-08 18:14:00,DRUG/NARCOTIC,POSSESSION OF MARIJUANA,Friday,SOUTHERN,"ARREST, BOOKED",13TH ST / FOLSOM ST,-122.415578,37.769591
372460,2010-03-08 13:45:00,LARCENY/THEFT,"LOST PROPERTY, GRAND THEFT",Monday,INGLESIDE,NONE,0 Block of PHELAN AV,-122.452290,37.725693


In [12]:
data_sample.shape

(5000, 9)

# Preprocessing

In [13]:
p_data = preprocess(data_sample)

In [14]:
p_data.shape

(5000, 8)

In [15]:
p_data.head()

,Category,DayOfWeek,PdDistrict,X,Y,Year,Week,Hour
579003,20,6,4,122.424437,37.775740,2007,10,7
214827,20,2,7,122.403405,37.775421,2012,26,22
600439,33,2,3,122.435150,37.761760,2006,44,14
164619,6,0,7,122.415578,37.769591,2013,10,18
372460,15,1,2,122.452290,37.725693,2010,10,13


# Feature Selection

In [16]:
p_final = extract_features(p_data, p_data.Category, chi2, threshold=.8, kBest=5)

In [17]:
p_final.head()

,Category,DayOfWeek,PdDistrict,Week,Hour
579003,20,6,4,10,7
214827,20,2,7,26,22
600439,33,2,3,44,14
164619,6,0,7,10,18
372460,15,1,2,10,13


In [18]:
p_final.shape

(5000, 5)

# kFolds Cross Validation

In [19]:
results = model_kfolds(p_final, "Category", models)
    
    
#return performance metrics 
results = [{x[0] : {'accuracy' : x[1]['test_accuracy'].mean(),
                    'precision' : x[1]['test_precision_micro'].mean(),
                    'recall' : x[1]['test_recall_micro'].mean(),
                    'f1' : x[1]['test_f1_micro'].mean()}} for x in results]

In [20]:
results

[{'decision tree': {'accuracy': 0.9992118226600984,
   'f1': 0.9992118226600984,
   'precision': 0.9992118226600984,
   'recall': 0.9992118226600984}}]